In [1]:
import os, sys
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
from hist.intervals import clopper_pearson_interval
def get_yerr(num,den):
    return abs(clopper_pearson_interval(num.view(), den.view()) - num.view()/den.view())

# we suppress ROOT warnings where our input ROOT tree has duplicate branches - these are handled correctly.
import warnings
warnings.filterwarnings("ignore", message="Found duplicate branch ")
np.seterr(divide='ignore', invalid='ignore')

from hist.intervals import clopper_pearson_interval
def get_yerr(num,den):
    return abs(clopper_pearson_interval(num.view(), den.view()) - np.nan_to_num(num.view()/den.view(), False, 1))

In [2]:
# you can load the output!
import pickle
with open('/eos/uscms/store/user/fmokhtar/boostedhiggs/lumiv3_2018/lumi_set.pkl', 'rb') as f:
    lumi_set = pickle.load(f)

In [3]:
lumi_set.keys()

dict_keys(['EGamma_Run2018A', 'EGamma_Run2018B', 'EGamma_Run2018C', 'EGamma_Run2018D', 'SingleMuon_Run2018A', 'SingleMuon_Run2018B', 'SingleMuon_Run2018C', 'SingleMuon_Run2018D'])

In [4]:
# combine the sets from the different datasets
lumis = {}

for i, dataset in enumerate(lumi_set.keys()):

    if "Muon" in dataset:
        ch = "mu"
    else:
        ch = "ele"
    print(dataset, ch)
    
    if ch not in lumis.keys():
        lumis[ch] = lumi_set[dataset]
    else:
        lumis[ch] = lumis[ch] | lumi_set[dataset]

# convert the set to a numpy 2d-array
for ch in ["ele", "mu"]:
    lumis[ch] = np.array(list(lumis[ch]))

EGamma_Run2018A ele
EGamma_Run2018B ele
EGamma_Run2018C ele
EGamma_Run2018D ele
SingleMuon_Run2018A mu
SingleMuon_Run2018B mu
SingleMuon_Run2018C mu
SingleMuon_Run2018D mu


In [6]:
from coffea.lumi_tools import LumiData, LumiMask, LumiList
from coffea.util import numpy as np

# make LumiList object
lumi_list = {}
for ch in ["ele", "mu"]:
    lumi_list[ch] = LumiList(runs=lumis[ch][:, 0], lumis=lumis[ch][:, 1])

    # this csv was made using brilcalc and the GoldenJson2017
    # refer to https://github.com/CoffeaTeam/coffea/blob/52e102fce21a3e19f8c079adc649dfdd27c92075/coffea/lumi_tools/lumi_tools.py#L20
    lumidata = LumiData("../lumi/lumi2018.csv")
    print(ch, lumidata.get_lumi(lumi_list[ch]))
    
    # 41480 is what we are using in the processor for 2017

ele 59816.22907800278
mu 59781.96029993873
